In [1]:
import pandas as pd
import pymysql
import numpy as np
import sqlalchemy
from sqlalchemy.sql import text as querytoText
import os

In [2]:
passwd = '1234Q1@'

url = sqlalchemy.engine.URL.create(
	drivername="mysql+pymysql",
    username='KMS',
    password=passwd,
    host='175.122.58.102',
    port=33063,
    database='ProjectDB',
)

engine = sqlalchemy.create_engine(url)
conn = engine.connect()

In [3]:
dfDict = {}
for file in os.listdir('DATA'):
    df_tmp =  pd.read_csv(f'DATA/{file}', encoding='euc-kr', dtype='object')
    dfDict[file.split('.')[0]] = df_tmp

In [4]:

for name, dfin in dfDict.items():
    dfin.dropna(axis=1, how='all', inplace=True)
    dfDict[name] = dfDict[name].apply(lambda x : x.str.replace(" 년", ""))
    if name != '개수면적':
        dfDict[name][dfDict[name].columns[-2:]] = dfDict[name][dfDict[name].columns[-2:]].apply(lambda x : pd.to_numeric(x, errors='coerce', downcast='integer'))
        dfDict[name].fillna(0, inplace=True)
        # change dtype to np.uint32
        dfDict[name][dfDict[name].columns[-2:]] = dfDict[name][dfDict[name].columns[-2:]].astype('uint32')

    else:
        dfDict[name][dfDict[name].columns[-3:]] = dfDict[name][dfDict[name].columns[-3:]].apply(lambda x : pd.to_numeric(x, errors='coerce', downcast='integer'))
        dfDict[name].fillna(0, inplace=True)
        # change dtype to np.uint32
        dfDict[name][dfDict[name].columns[-3:]] = dfDict[name][dfDict[name].columns[-3:]].astype('uint32')

In [5]:
for name, dfin in dfDict.items():
    print(dfin.dtypes)

소재지(시군구)별    object
교육과정별        object
시점           uint32
시설수[개]       uint32
면적[㎡]        uint32
dtype: object
[STCODE_11]시도별      object
시도별                 object
[STCODE_12]설립주체별    object
설립주체별               object
[STCODE_13]학교현황별    object
학교현황별               object
시점                  uint32
고등학교 개황             uint32
dtype: object
소재지(시군구)별(1)    object
소재지(시군구)별(2)    object
항목              object
시점              uint32
데이터             uint32
dtype: object
시도별(1)       object
설립주체별(1)     object
유치원현황별(1)    object
유치원현황별(2)    object
시점           uint32
데이터          uint32
dtype: object
[STCODE_11]시도별      object
시도별                 object
[STCODE_12]설립주체별    object
설립주체별               object
[STCODE_13]학교현황별    object
학교현황별               object
시점                  uint32
중학교 개황              uint32
dtype: object
시도별(1)      object
설립주체별(1)    object
학교현황별(1)    object
학교현황별(2)    object
시점          uint32
데이터         uint32
dtype: object


In [6]:
for name, dfin in dfDict.items():
    header = list(dfDict[name].columns)
    for idx, data in enumerate(header):
        if data == '시점':
            header[idx] = 'year'
        if (data.find("시도") >= 0 or data.find("시군구") >= 0) and data.find("STCODE") < 0:
            header[idx] = '행정구역' + str(idx)
  
    
    
    dfDict[name].columns = header
    
    

In [7]:
for name, dfin in dfDict.items():
    conn.execute(querytoText(f"DROP TABLE IF EXISTS KMS_{name}"))
    conn.commit()
    dfDict[name].to_sql(f"KMS_{name}", con=conn, if_exists='replace', index=False)
    keylist = list(dfDict[name].columns)[:-1] if name != '개수면적' else list(dfDict[name].columns)[:-2]
    for col in keylist[:-1]:
        varcharQuery = f"""
        alter table KMS_{name}
        modify `{col}` varchar(30) null;
        """
        yearQuery =  f"""
        alter table KMS_{name}
        modify `year` smallint null;"""
        conn.execute(querytoText(varcharQuery))
        conn.execute(querytoText(yearQuery))
        conn.commit()
    query = f'''
    alter table KMS_{name}
    add constraint KMS_{name}_pk
        primary key (`{'`, `'.join(keylist)}`);
    '''
    conn.execute(querytoText(query))  if name != '개수면적' else ""
    conn.commit()

In [8]:
pd.Series(dfDict['미집행현황'][dfDict['미집행현황'].columns[0]].unique().tolist(), name='시도별').to_sql('KMS_시도별', con=conn, if_exists='replace',index=False)

18

In [9]:
pd.Series(dfDict['유치원']['year'].unique(), name='year').to_sql('KMS_year', con=conn, if_exists='replace',index=False)

10

In [12]:
pd.read_sql("select distinct 행정구역0 from KMS_유치원", con=conn)

,행정구역0
0,강원특별자치도
1,경기도
2,경상남도
3,경상북도
4,광주광역시
5,대구광역시
6,대전광역시
7,부산광역시
8,서울특별시
9,세종특별자치시


In [11]:
for col in ['KMS_유치원', 'KMS_초등학교']:
    conn.execute(querytoText(f"update {col} set 행정구역0 = '전국' where 행정구역0 = '총계'"))

In [19]:
pdwith시군 = pd.read_sql(f"select distinct 행정구역0, 행정구역1 from KMS_미집행현황 where 행정구역1 not like '소계'", con=conn)

In [21]:
pdwith시군.to_sql('KMS_행정구역_시군포함', con=conn, if_exists='replace')

157

In [29]:
for table in ['KMS_유치원', 'KMS_초등학교']:
    conn.execute(querytoText(f'''
    update {table}
    set 행정구역0 = '강원도'
    where 행정구역0 like '강원%'
    '''))
conn.commit()